# 목표
---
> 해당 날짜 별, 노선 및 버스 ID가 주어진 상황에서 현재 정류소의 이름과 현재 정류장 도착 시간을 이용하여,다음 정류장에 도착할 때까지 걸리는 시간을 예측

*   next_arrive_time : 클래스
*  나머지 column : 속성



# EDA
---


In [2]:
import pandas as pd

## 데이터 불러오기 및 확인


In [3]:
train=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Kaggle_Study/bus/train.csv')
test=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Kaggle_Study/bus/test.csv')

### pd.DataFrame.head()
* dataframe의 앞에 5줄을 확인할 수 있음.
* 끝의 5개 행은 **pd.DataFrame.tail()**

In [4]:
train.head()

,id,date,route_id,vh_id,route_nm,now_latitude,now_longitude,now_station,now_arrive_time,distance,next_station,next_latitude,next_longitude,next_arrive_time
0,0,2019-10-15,405136001,7997025,360-1,33.456267,126.551750,제주대학교입구,06시,266.0,제대마을,33.457724,126.554014,24
1,1,2019-10-15,405136001,7997025,360-1,33.457724,126.554014,제대마을,06시,333.0,제대아파트,33.458783,126.557353,36
2,2,2019-10-15,405136001,7997025,360-1,33.458783,126.557353,제대아파트,06시,415.0,제주대학교,33.459893,126.561624,40
3,3,2019-10-15,405136001,7997025,360-1,33.479705,126.543811,남국원(아라방면),06시,578.0,제주여자중고등학교(아라방면),33.484860,126.542928,42
4,4,2019-10-15,405136001,7997025,360-1,33.485662,126.494923,도호동,07시,374.0,은남동,33.485822,126.490897,64


### pd.DataFrame.info()
* 각 column 들의 정보 확인 가능
* Null의 수, Dtype 등을 확인할 수 있음


> 결측치는 없는디 문자열이 존재

**문자열인 데이터**
*   date
*   route_nm
* now_station
* now_arrive_time
* next_station


In [5]:
train.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 210457 entries, 0 to 210456
Data columns (total 14 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   id                210457 non-null  int64  
 1   date              210457 non-null  object 
 2   route_id          210457 non-null  int64  
 3   vh_id             210457 non-null  int64  
 4   route_nm          210457 non-null  object 
 5   now_latitude      210457 non-null  float64
 6   now_longitude     210457 non-null  float64
 7   now_station       210457 non-null  object 
 8   now_arrive_time   210457 non-null  object 
 9   distance          210457 non-null  float64
 10  next_station      210457 non-null  object 
 11  next_latitude     210457 non-null  float64
 12  next_longitude    210457 non-null  float64
 13  next_arrive_time  210457 non-null  int64  
dtypes: float64(5), int64(4), object(5)
memory usage: 22.5+ MB


In [6]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 91774 entries, 0 to 91773
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               91774 non-null  int64  
 1   date             91774 non-null  object 
 2   route_id         91774 non-null  int64  
 3   vh_id            91774 non-null  int64  
 4   route_nm         91774 non-null  object 
 5   now_latitude     91774 non-null  float64
 6   now_longitude    91774 non-null  float64
 7   now_station      91774 non-null  object 
 8   now_arrive_time  91774 non-null  object 
 9   distance         91774 non-null  float64
 10  next_station     91774 non-null  object 
 11  next_latitude    91774 non-null  float64
 12  next_longitude   91774 non-null  float64
dtypes: float64(5), int64(3), object(5)
memory usage: 9.1+ MB


### pd.DataFrame.shape()
* (row,column)을 표현
* (해당 데이터의 수, 정보의 종류)로 이해할 수 있음.

In [7]:
print(train.shape)
print(test.shape)

(210457, 14)
(91774, 13)


### pd.DataFrame.descirbe()
* count : 데이터 수
* mean : 데이터 평균
* std : 표준 편차
* min : 최소
* max : 최대
* 25,50,75 : 해당 백분율에 있는 데이터 수

In [8]:
train.describe()

,id,route_id,vh_id,now_latitude,now_longitude,distance,next_latitude,next_longitude,next_arrive_time
count,210457.000000,2.104570e+05,2.104570e+05,210457.000000,210457.000000,210457.000000,210457.000000,210457.000000,210457.000000
mean,105228.000000,4.052491e+08,7.988694e+06,33.434528,126.603451,490.256100,33.434711,126.603687,85.380824
std,60753.847139,9.132404e+04,6.774077e+03,0.102350,0.123961,520.563932,0.102224,0.123838,85.051170
min,0.000000,4.051360e+08,7.983000e+06,33.244382,126.473300,97.000000,33.244382,126.473300,6.000000
25%,52614.000000,4.051365e+08,7.983093e+06,33.325283,126.523900,291.000000,33.325283,126.524550,44.000000
50%,105228.000000,4.053201e+08,7.983431e+06,33.484667,126.551050,384.000000,33.484860,126.551050,66.000000
75%,157842.000000,4.053201e+08,7.997041e+06,33.500197,126.650322,542.000000,33.500228,126.650322,102.000000
max,210456.000000,4.053281e+08,7.997124e+06,33.556167,126.935188,7461.000000,33.556167,126.935188,2996.000000


### DataFrame.drop_duplicates()
> 중복되는 행을 제거하는 메서드



In [9]:
train[['route_id','vh_id']].drop_duplicates()

,route_id,vh_id
0,405136001,7997025
181,405136001,7997034
381,405136001,7997035
539,405136001,7997040
717,405136001,7997044
...,...,...
20226,405136522,7997004
26037,405320116,7983225
27098,405320122,7983067
45506,405328102,7983401


### pd. DataFrame.groupby()
* 원하는 열들을 엮어 평균 등 다양한 통계량을 확인할 수 있음.
> 405320122 경로에서 가장 적고, 405136521에서 가장 많다. 그렇다면 많은 부분은 번화가 주변, 아닌 부분은 외곽에 있지 않을까?

In [10]:
#경로 아이디를 기준으로 다음 도착 시간의 평균 시간을 정리한 표 
train[['route_id','next_arrive_time']].groupby('route_id').mean() 

,next_arrive_time
route_id,
405136001,92.063688
405136002,108.270852
405136007,94.172947
405136012,111.322633
405136521,114.673939
405136522,101.339188
405320111,68.901710
405320112,67.602392
405320113,68.403649


## 시각화
---
> 데이터에 위도, 경도 자료가 있기 때문에 지도에 시각화하는 게 중요함.    




### 한글

In [11]:
## matplotlib 사용시 한국어 나오게 하는 코드

''' !apt -qq -y install fonts-nanum > /dev/null

import matplotlib.font_manager as fm
import matplotlib.pyplot as plt

fontpath = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'

font = fm.FontProperties(fname=fontpath, size=9)

fm._rebuild()

# 그래프에 retina display 적용

%config InlineBackend.figure_format = 'retina'

# Colab 의 한글 폰트 설정

plt.rc('font', family='NanumBarunGothic') '''

" !apt -qq -y install fonts-nanum > /dev/null\n\nimport matplotlib.font_manager as fm\nimport matplotlib.pyplot as plt\n\nfontpath = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'\n\nfont = fm.FontProperties(fname=fontpath, size=9)\n\nfm._rebuild()\n\n# 그래프에 retina display 적용\n\n%config InlineBackend.figure_format = 'retina'\n\n# Colab 의 한글 폰트 설정\n\nplt.rc('font', family='NanumBarunGothic') "

### folium.Map 맛보기
> 지리 정보 시각화 프로그램 **folium**   
* location : 해당 위도, 경도를 중심으로 map을 보여줌.
* zoom_start : 확대 크기를 설정 (기본 10)
* tiles : 맵의 default를 변경


In [12]:
#지리 정보 시각화 프로그램
import folium 

In [13]:
coord = [35.1559361, 129.1384361]
map_osm=folium.Map(location=coord) #coord라는 위도, 경도를 중심으로 하는 Map
map_osm

In [14]:
map_osm=folium.Map(location=coord,zoom_start=12) #zoom_start: 확대 크기를 설정해줌
map_osm 

In [15]:
map_osm=folium.Map(location=coord,tiles='Stamen Terrain') #맵의 default 배경 변경
map_osm

#### folium.Marker
> 넣을 때 add_to(맵 변수)사용
* popup : popup할 때 나오는 글자 형성
* tooltip : 물방울 모양으로 뜨는 거
* icon : popup에서 물방울 모양에 다양한 로고 넣음



In [16]:
coord = [35.1559361, 129.1384361]
map_osm=folium.Map(location=coord,zoom_start=12)
folium.Marker(coord,popup='부산대학교',tooltip='부산대학교 tootltip').add_to(map_osm)
map_osm.save('index.html')
map_osm

In [17]:
map_osm=folium.Map(location=coord,zoom_start=12)
folium.Marker(coord,popup='부산대학교',icon=folium.Icon(icon='home')).add_to(map_osm)
map_osm

#### folium.Circle && folium.CircleMarker
---
* Circle : 지정한 위경에서 원을 그려주며, 크기는 고정됨.
* CircleMarker : 지정한 위경도에서 해상도에 따라 크기를 맞춰주면서 원 크기가 변경됨.
> radius, color, fill_color 등 다양한 색 지정이 가능함.

In [18]:
map_osm = folium.Map(location=[37.541, 126.986], zoom_start=11)
folium.Marker([37.566345, 126.977893], popup='서울특별시청', icon=folium.Icon(color='red',icon='info-sign')).add_to(map_osm)
folium.Circle([37.4600, 126.9519], tooltip='서울대학교', radius=3000, color="green").add_to(map_osm)
folium.CircleMarker([37.5662, 126.9386], radius=10, popup='연세대학교',color="black", fill_color="crimson").add_to(map_osm)
map_osm

#### folium.RegularPolygonMarker
---
> 다각형의 Marker를 형성할 수 있음. 
* color, number_of_sides,radius 등 다양한 색지정, 꼭짓점 수, 반지름 등 가능함.

In [19]:
map_osm = folium.Map(location=[37.541, 126.986], zoom_start=11)
folium.RegularPolygonMarker([37.566345, 126.977893], popup='서울특별시청', fill_color='red',color='red', number_of_sides=3, radius=30, fill_opacity=0.2).add_to(map_osm)
folium.RegularPolygonMarker([37.4600, 126.9519], popup='서울대학교', fill_color='green',color='green', number_of_sides=4, radius=10, fill_opacity=0.2).add_to(map_osm)
folium.RegularPolygonMarker([37.5662, 126.9386], popup='연세대학교',fill_color='blue',color='blue', number_of_sides=6, radius=10).add_to(map_osm)
map_osm

### folium으로 EDA
> '경로마다 번화가냐 아니냐에 따라 걸리는 시간에 차이가 있을 것이다.'라는 가설을 세운 후 접근

**지도에 필요한 정보**
* 해당 위치의 위도와 경도
* 정류장 이름


#### 배차 간격이 짧은 곳은 교통 체증이 없는 외곽 쪽이 아닐까

In [20]:
map_data = train[['route_id','now_latitude','now_longitude','now_station']]
map_bus_route = map_data[map_data['route_id'] == 405320122].drop_duplicates("now_station")

In [21]:
map_osm = folium.Map(location=[33.4134, 126.5190], zoom_start = 10.5)

In [22]:
for item in map_bus_route.index: #dataframe.index : 색인의 정보를 추출
    lat = map_bus_route.loc[item,'now_latitude'] #myframe.loc[[행라벨,][열라벨, ]] 해당 행과 열의 정보를 읽어옴.
    long = map_bus_route.loc[item,'now_longitude']
    
    folium.Marker([lat,long], popup = map_bus_route.loc[item,'now_station'],
                      icon = folium.Icon(color = 'red', icon = 'info-sign')
                      ).add_to(map_osm)

map_osm
                    

#### 배차 간격이 긴 곳은 교통 체증이 많은 번화가 쪽이 아닐까

In [23]:
#405136521
map_data = train[['route_id','now_latitude','now_longitude','now_station']]
map_bus_route=map_data[map_data['route_id']==405136521].drop_duplicates('now_station')

In [24]:
map_osm = folium.Map(location=[33.4134, 126.5190], zoom_start = 10.5)

In [25]:
for item in map_bus_route.index :
  lat=map_bus_route.loc[item,'now_latitude']
  long=map_bus_route.loc[item,'now_longitude']

  folium.Marker([lat,long],popup=map_bus_route.loc[item,'now_station'],
                icon=folium.Icon(color='red',icon='info-sign')).add_to(map_osm)

map_osm

### seaborn heatmap 함수


> distance와 0.44로 가장 높은 상관관계를 보인다.하지만 상세한 가설은 위도 경도를 나타낸 맵으로 세우는 게 좋을 거 같다.


*   now_latitude와 new_latitude가 같다 
*   now_latitude와 new_latitude가 같다






In [26]:
'''import seaborn as sns

sns.heatmap(train.corr(),annot=True)'''

'import seaborn as sns\n\nsns.heatmap(train.corr(),annot=True)'

# 데이터 전처리
---
* 결측값 처리 ❌
* 모델이 학습 가능한 형태로의 변화 ⭕

**문자열인 데이터 중 형태 변화가 필요한 것**
*   date
*   route_nm => 아래 결과에 따라 route_id와 route_nm은 동일하게 봐도 될듯.
* now_arrive_time => 출퇴근 시간에 따른 변화가 있을 수 있다!



In [30]:
check=train[train['route_id']==405136001]
check

,id,date,route_id,vh_id,route_nm,now_latitude,now_longitude,now_station,now_arrive_time,distance,next_station,next_latitude,next_longitude,next_arrive_time
0,0,2019-10-15,405136001,7997025,360-1,33.456267,126.551750,제주대학교입구,06시,266.0,제대마을,33.457724,126.554014,24
1,1,2019-10-15,405136001,7997025,360-1,33.457724,126.554014,제대마을,06시,333.0,제대아파트,33.458783,126.557353,36
2,2,2019-10-15,405136001,7997025,360-1,33.458783,126.557353,제대아파트,06시,415.0,제주대학교,33.459893,126.561624,40
3,3,2019-10-15,405136001,7997025,360-1,33.479705,126.543811,남국원(아라방면),06시,578.0,제주여자중고등학교(아라방면),33.484860,126.542928,42
4,4,2019-10-15,405136001,7997025,360-1,33.485662,126.494923,도호동,07시,374.0,은남동,33.485822,126.490897,64
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197668,197668,2019-10-28,405136001,7997124,360-1,33.485822,126.490897,은남동,21시,267.0,제원아파트,33.487531,126.489892,44
197669,197669,2019-10-28,405136001,7997124,360-1,33.476260,126.481417,제주고등학교/중흥S클래스,21시,244.0,부영아파트,33.476900,126.483133,86
197670,197670,2019-10-28,405136001,7997124,360-1,33.478850,126.488350,대림2차아파트,21시,334.0,연동대림1차아파트,33.480700,126.489933,50
197671,197671,2019-10-28,405136001,7997124,360-1,33.487317,126.496617,연동주민센터,22시,455.0,제주도청 신제주로터리,33.491433,126.496783,58


In [35]:
check[check['route_nm']=='360-1']

,id,date,route_id,vh_id,route_nm,now_latitude,now_longitude,now_station,now_arrive_time,distance,next_station,next_latitude,next_longitude,next_arrive_time
0,0,2019-10-15,405136001,7997025,360-1,33.456267,126.551750,제주대학교입구,06시,266.0,제대마을,33.457724,126.554014,24
1,1,2019-10-15,405136001,7997025,360-1,33.457724,126.554014,제대마을,06시,333.0,제대아파트,33.458783,126.557353,36
2,2,2019-10-15,405136001,7997025,360-1,33.458783,126.557353,제대아파트,06시,415.0,제주대학교,33.459893,126.561624,40
3,3,2019-10-15,405136001,7997025,360-1,33.479705,126.543811,남국원(아라방면),06시,578.0,제주여자중고등학교(아라방면),33.484860,126.542928,42
4,4,2019-10-15,405136001,7997025,360-1,33.485662,126.494923,도호동,07시,374.0,은남동,33.485822,126.490897,64
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197668,197668,2019-10-28,405136001,7997124,360-1,33.485822,126.490897,은남동,21시,267.0,제원아파트,33.487531,126.489892,44
197669,197669,2019-10-28,405136001,7997124,360-1,33.476260,126.481417,제주고등학교/중흥S클래스,21시,244.0,부영아파트,33.476900,126.483133,86
197670,197670,2019-10-28,405136001,7997124,360-1,33.478850,126.488350,대림2차아파트,21시,334.0,연동대림1차아파트,33.480700,126.489933,50
197671,197671,2019-10-28,405136001,7997124,360-1,33.487317,126.496617,연동주민센터,22시,455.0,제주도청 신제주로터리,33.491433,126.496783,58


## pd.Series.str.slice()
---
* start : 기본으로는 처음부터 시작함.
* stop : 어디까지 슬라이스할지 결정함.

**astype()**
> 자료형을 변경해주는 함수

In [ ]:
train['now_arrive_time']=train['now_arrive_time'].str.slice(stop=-1).astype('int')

In [48]:
train['now_arrive_time']

0          6
1          6
2          6
3          6
4          7
          ..
210452    21
210453    21
210454    21
210455    21
210456    21
Name: now_arrive_time, Length: 210457, dtype: int64

In [50]:
test['now_arrive_time']=test['now_arrive_time'].str.slice(stop=-1).astype('int')
test['now_arrive_time']

0         7
1         7
2         8
3         8
4         8
         ..
91769    20
91770    20
91771    20
91772    20
91773    20
Name: now_arrive_time, Length: 91774, dtype: int64

# 모델링
---


1.   feature와 target 설정
2.   train과 test 별로 x,y 데이터를 설정
3. 사용할 모델 채택
4. 모델 학습
5. 모델 예측

> 이외에도 부가적으로 다양한 feature를 설정,하이퍼파라미터 튜닝, 교차 검증 등을 한다면 정확한 예측이 가능하다.



## 변수 선택 및 모델 구축 
---
> feature로 설정할 column을 선정하고, 학습셋과 훈련셋 별로 x와 y 선정한다. 사용할 모델을 선정한다.

In [52]:
features=['now_latitude','now_longitude','now_arrive_time','distance']
target=['next_arrive_time']

In [53]:
X_train,X_test,y_train=train[features],test[features],train[target]

## 모델 학습 및 검증
> 특히 이번 실습에서는 다양한 모델을 이용하여 그 때마다의 학습과 예측결과를 만들어내는 프로그램을 만든다.

In [55]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
import lightgbm as lgb

In [60]:
model_dict={
    'linear':LinearRegression(),
    'rf':RandomForestRegressor(random_state=0,n_jobs=-1),
    'lgbm':lgb.LGBMRegressor(random_state=0)
}

In [67]:
model_result={}
for key in model_dict.keys(): #dict.keys : 딕셔너리의 key값을 정리하는 함수
  print('#### 훈련 중 ####')
  model_dict[key].fit(X_train,y_train)

  print('#### 예측 중 ####')
  model_result[key]=model_dict[key].predict(X_test)

#### 훈련 중 ####
#### 예측 중 ####
#### 훈련 중 ####


<ipython-input-67-2e07274b702e>:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model_dict[key].fit(X_train,y_train)


#### 예측 중 ####
#### 훈련 중 ####
#### 예측 중 ####


In [79]:
for key in model_result.keys():
  print(model_result)

{'linear': array([[ 80.75162733],
       [ 86.59637449],
       [ 96.03730695],
       ...,
       [ 93.13399549],
       [ 72.93764136],
       [103.28646517]]), 'rf': array([ 35.36650507,  45.12237491, 135.88134374, ..., 126.37972036,
        31.84467172,  88.46184286]), 'lgbm': array([ 42.12400643,  60.24878305, 120.81241189, ..., 105.81387439,
        30.82952485, 111.66787953])}
{'linear': array([[ 80.75162733],
       [ 86.59637449],
       [ 96.03730695],
       ...,
       [ 93.13399549],
       [ 72.93764136],
       [103.28646517]]), 'rf': array([ 35.36650507,  45.12237491, 135.88134374, ..., 126.37972036,
        31.84467172,  88.46184286]), 'lgbm': array([ 42.12400643,  60.24878305, 120.81241189, ..., 105.81387439,
        30.82952485, 111.66787953])}
{'linear': array([[ 80.75162733],
       [ 86.59637449],
       [ 96.03730695],
       ...,
       [ 93.13399549],
       [ 72.93764136],
       [103.28646517]]), 'rf': array([ 35.36650507,  45.12237491, 135.88134374, ..., 126

In [70]:
submission=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Kaggle_Study/bus/submission_제출양식.csv')

In [81]:
#submission과 같은 파일에 저장해야하므로 모두 copy로 복사함
linear_sub = submission.copy()
rf_sub = submission.copy()
lgbm_sub = submission.copy()

In [82]:
linear_sub

,id,next_arrive_time
0,210457,0
1,210458,0
2,210459,0
3,210460,0
4,210461,0
...,...,...
91769,302350,0
91770,302351,0
91771,302352,0
91772,302353,0


In [83]:
linear_sub['next_arrive_time']=model_result['linear']
rf_sub['next_arrive_time']=model_result['rf']
lgbm_sub['next_arrive_time']=model_result['lgbm']

In [84]:
rf_sub

,id,next_arrive_time
0,210457,35.366505
1,210458,45.122375
2,210459,135.881344
3,210460,57.375342
4,210461,153.307466
...,...,...
91769,302350,106.596700
91770,302351,76.308277
91771,302352,126.379720
91772,302353,31.844672


In [86]:
linear_sub.to_csv('linear_sub.csv',index=False)
rf_sub.to_csv('rf_sub.csv',index=False)
lgbm_sub.to_csv('lgbm_sub.csv',index=False)

# 결과
---
> 단순한 모델인 linear 모델보다 lgbm, randomforest 순으로 높은 점수가 나왔다. 파라미터를 변경하면서 모델을 비교한 따릉이와, 교차검증 등을 진행한 영화 코드를 보며 학습률 높이는 방법을 찾아봐야겠다.